# Tarea 2: Conteo Aproximado con MCMC
## Aplicado a los modelos Hard-Core y q-coloraciones

**Curso:** Cadenas de Markov y Aplicaciones (2025-II)  
**Profesor:** Freddy Hernández-Romero  

**Integrantes del grupo:**
- Sergio Andrés Díaz Vera (seadiazve@unal.edu.co)
- Julián Mateo Espinosa Ospina (juespinosao@unal.edu.co)

**Fecha de entrega:** Noviembre 2025

## Introducción

En esta tarea implementamos algoritmos de conteo aproximado basados en Cadenas de Markov Monte Carlo (MCMC) para dos problemas fundamentales en teoría de grafos:

1. **q-Coloraciones**: Asignaciones de q colores a los vértices de un grafo tal que vértices adyacentes tienen colores diferentes.
2. **Modelo Hard-Core**: Configuraciones donde ningún par de vértices adyacentes están simultáneamente ocupados.

Basamos nuestra implementación en el **Teorema 9.1**, que establece la existencia de un esquema de aproximación polinomial aleatorizado para estos problemas.

In [2]:
# Importaciones necesarias
import sys
import os

# Agregar el directorio padre al path
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
%matplotlib inline

# Importar implementación mejorada desde src
from src.mcmc_counting import (
    LatticeGraph, 
    QColoringApproximation, 
    HardCoreApproximation,
    exact_chromatic_polynomial,
    exact_hardcore_count,
    run_q_coloring_experiment,
    run_hardcore_experiment
)

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 1. Aproximación del Número de q-Coloraciones

### 1.a) Implementación basada en el Teorema 9.1

El Teorema 9.1 establece que para grafos con grado máximo $d$ y $q > 2d^*$, existe un esquema de aproximación con:

- **Número de simulaciones**: $\frac{48d^2k^3}{\varepsilon^2}$
- **Pasos del Gibbs Sampler**: $k\left(\frac{2\log(k)+\log(\varepsilon^{-1})+\log(8)}{\log\frac{q}{q-1}} + 1\right)$

donde $k$ es el número de vértices y $\varepsilon$ es la precisión deseada.

In [3]:
# Parámetros del experimento según el documento
K_range = range(3, 21)  # 3 ≤ K ≤ 20
q_range = range(2, 16)  # 2 ≤ q ≤ 15
epsilon_values = [0.5, 0.2, 0.1]  # Valores de precisión

print("Configuración de experimentos:")
print(f"  • Tamaños de lattice K: {list(K_range)}")
print(f"  • Número de colores q: {list(q_range)}")
print(f"  • Valores de epsilon: {epsilon_values}")

Configuración de experimentos:
  • Tamaños de lattice K: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
  • Número de colores q: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
  • Valores de epsilon: [0.5, 0.2, 0.1]


In [4]:
# Experimentos de muestra
print("\n=== Experimentos de muestra ===")
sample_results = []

# Casos representativos
sample_cases = [
    (3, 9, 0.1),
    (4, 9, 0.2),
    (5, 10, 0.1),
]

for K, q, eps in sample_cases:
    print(f"\nEjecutando: K={K}, q={q}, ε={eps}")
    result = run_q_coloring_experiment(K, q, eps, verbose=True)
    sample_results.append(result)


=== Experimentos de muestra ===

Ejecutando: K=3, q=9, ε=0.1

=== Conteo Aproximado de q-Coloraciones ===
Lattice: 3 × 3
q = 9, d = 4, k = 9
ε = 0.1
Simulaciones por factor: 1000
Tiempo de mezcla: 680
Burn-in: 100
  Progreso: 1000/5000 muestras
  Progreso: 2000/5000 muestras
  Progreso: 3000/5000 muestras
  Progreso: 4000/5000 muestras
  Progreso: 5000/5000 muestras

✓ Completado en 32.29 segundos
Estimación: 1.95e+06

Ejecutando: K=4, q=9, ε=0.2

=== Conteo Aproximado de q-Coloraciones ===
Lattice: 4 × 4
q = 9, d = 4, k = 16
ε = 0.2
Simulaciones por factor: 1000
Tiempo de mezcla: 1271
Burn-in: 127
  Progreso: 1000/5000 muestras
  Progreso: 2000/5000 muestras
  Progreso: 3000/5000 muestras
  Progreso: 4000/5000 muestras
  Progreso: 5000/5000 muestras

✓ Completado en 57.06 segundos
Estimación: 1.53e+11

Ejecutando: K=5, q=10, ε=0.1

=== Conteo Aproximado de q-Coloraciones ===
Lattice: 5 × 5
q = 10, d = 4, k = 25
ε = 0.1
Simulaciones por factor: 1000
Tiempo de mezcla: 2593
Burn-in: 259

In [ ]:
# Ejecutar experimentos sistemáticos
print("\n=== Ejecutando experimentos completos ===")

all_results_q = []
K_subset = [3, 4, 5, 6, 8, 10]
q_subset = [9, 10, 12, 15]

total = len(K_subset) * len(q_subset) * len(epsilon_values)
with tqdm(total=total, desc="Experimentos q-coloraciones") as pbar:
    for K in K_subset:
        for q in q_subset:
            for eps in epsilon_values:
                result = run_q_coloring_experiment(K, q, eps)
                if result['valid']:
                    all_results_q.append(result)
                pbar.update(1)

df_q = pd.DataFrame(all_results_q)
print(f"\n{len(all_results_q)} experimentos válidos completados")

# Mostrar estadísticas
print("\n=== Resumen de parámetros utilizados ===")
for eps in epsilon_values:
    df_eps = df_q[df_q['epsilon'] == eps]
    if not df_eps.empty:
        print(f"\nε = {eps}:")
        print(f"  • Simulaciones promedio: {df_eps['num_simulations'].mean():.0f}")
        print(f"  • Tiempo de mezcla promedio: {df_eps['mixing_time'].mean():.0f} pasos")
        print(f"  • Tiempo ejecución promedio: {df_eps['elapsed_time'].mean():.2f} segundos")


=== Ejecutando experimentos completos ===


Experimentos q-coloraciones:  24%|██▎       | 17/72 [12:36<53:46, 58.66s/it]

### Visualización de Resultados para q-Coloraciones

In [ ]:
# Crear visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Estimaciones vs tamaño del lattice para diferentes q
ax1 = axes[0, 0]
for q in df_q['q'].unique():
    df_q_filtered = df_q[(df_q['q'] == q) & (df_q['epsilon'] == 0.1)]
    if not df_q_filtered.empty:
        ax1.semilogy(df_q_filtered['K'], df_q_filtered['estimate'], 
                    'o-', label=f'q={q}', markersize=8)
ax1.set_xlabel('Tamaño del Lattice (K)')
ax1.set_ylabel('log(Número de q-coloraciones)')
ax1.set_title('Crecimiento del Número de q-Coloraciones')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Impacto de epsilon en el tiempo de ejecución
ax2 = axes[0, 1]
for eps in epsilon_values:
    df_eps = df_q[(df_q['epsilon'] == eps) & (df_q['q'] == 10)]
    if not df_eps.empty:
        ax2.plot(df_eps['K'], df_eps['elapsed_time'], 
                'o-', label=f'ε={eps}', markersize=8)
ax2.set_xlabel('Tamaño del Lattice (K)')
ax2.set_ylabel('Tiempo de Ejecución (segundos)')
ax2.set_title('Impacto de ε en el Tiempo de Cómputo (q=10)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Número de simulaciones requeridas
ax3 = axes[1, 0]
df_01 = df_q[df_q['epsilon'] == 0.1]
if not df_01.empty:
    pivot_sim = df_01.pivot_table(values='num_simulations', 
                                  index='K', columns='q', aggfunc='mean')
    sns.heatmap(pivot_sim, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax3)
    ax3.set_title('Número de Simulaciones Requeridas (ε=0.1)')
    ax3.set_xlabel('q (número de colores)')
    ax3.set_ylabel('K (tamaño del lattice)')

# 4. Tiempo de mezcla
ax4 = axes[1, 1]
if not df_01.empty:
    pivot_mix = df_01.pivot_table(values='mixing_time', 
                                  index='K', columns='q', aggfunc='mean')
    sns.heatmap(pivot_mix, annot=True, fmt='.0f', cmap='Blues', ax=ax4)
    ax4.set_title('Tiempo de Mezcla del Gibbs Sampler (ε=0.1)')
    ax4.set_xlabel('q (número de colores)')
    ax4.set_ylabel('K (tamaño del lattice)')

plt.tight_layout()
plt.savefig('../resultados/q_coloraciones_analisis.png', dpi=150, bbox_inches='tight')
plt.show()

print("Gráficas guardadas en resultados/q_coloraciones_analisis.png")

### 1.b) Comparación con Conteo Exacto

Para lattices pequeños, comparamos nuestras aproximaciones con el conteo exacto usando el polinomio cromático.

In [ ]:
print("=== Comparación con Conteo Exacto ===")
print("\nComparando aproximaciones con valores exactos para lattices pequeños:")
print("-" * 70)
print(f"{'K':^5} {'q':^5} {'Exacto':^15} {'Aproximado':^15} {'Error (%)':^15}")
print("-" * 70)

comparison_results = []
comparison_cases = [
    (2, 5),
    (2, 7),
    (3, 9),
    (3, 10),
]

for K, q in comparison_cases:
    try:
        exact = exact_chromatic_polynomial(K, q)
        
        lattice = LatticeGraph(K)
        approx_obj = QColoringApproximation(lattice, q, epsilon=0.05)
        estimate, _ = approx_obj.approximate_count(verbose=False)
        
        error = abs(estimate - exact) / exact * 100 if exact > 0 else 0
        
        print(f"{K:^5} {q:^5} {exact:^15,} {estimate:^15.0f} {error:^15.2f}")
        
        comparison_results.append({
            'K': K,
            'q': q,
            'exact': exact,
            'approximate': estimate,
            'error_percent': error
        })
        
    except Exception as e:
        print(f"{K:^5} {q:^5} {'N/A':^15} {'N/A':^15} {'N/A':^15}")

print("-" * 70)

df_comparison = pd.DataFrame(comparison_results)
if not df_comparison.empty:
    print(f"\nError promedio: {df_comparison['error_percent'].mean():.2f}%")
    print(f"Error máximo: {df_comparison['error_percent'].max():.2f}%")
    print(f"Error mínimo: {df_comparison['error_percent'].min():.2f}%")

## 2. Aproximación del Número de Configuraciones del Modelo Hard-Core

El modelo Hard-Core representa configuraciones donde ningún par de vértices adyacentes pueden estar ocupados simultáneamente.

In [ ]:
# Experimentos de muestra para Hard-Core
print("\n=== Experimentos Hard-Core de muestra ===")

hc_sample_cases = [
    (3, 0.1),
    (4, 0.2),
    (5, 0.1),
]

hc_sample_results = []
for K, eps in hc_sample_cases:
    print(f"\nEjecutando Hard-Core: K={K}, ε={eps}")
    result = run_hardcore_experiment(K, eps, verbose=True)
    hc_sample_results.append(result)

In [ ]:
# Ejecutar experimentos sistemáticos para Hard-Core
print("\n=== Ejecutando experimentos Hard-Core completos ===")

all_results_hc = []
K_range_hc = [3, 4, 5, 6, 8, 10, 12, 15]

total_hc = len(K_range_hc) * len(epsilon_values)
with tqdm(total=total_hc, desc="Experimentos Hard-Core") as pbar:
    for K in K_range_hc:
        for eps in epsilon_values:
            result = run_hardcore_experiment(K, eps)
            all_results_hc.append(result)
            pbar.update(1)

df_hc = pd.DataFrame(all_results_hc)
print(f"\n{len(all_results_hc)} experimentos Hard-Core completados")

# Mostrar estadísticas
print("\n=== Resumen de parámetros Hard-Core ===")
for eps in epsilon_values:
    df_eps_hc = df_hc[df_hc['epsilon'] == eps]
    print(f"\nε = {eps}:")
    print(f"  • Simulaciones promedio: {df_eps_hc['num_simulations'].mean():.0f}")
    print(f"  • Tiempo de mezcla promedio: {df_eps_hc['mixing_time'].mean():.0f} pasos")
    print(f"  • Partículas promedio: {df_eps_hc['avg_particles'].mean():.2f}")
    print(f"  • Tiempo ejecución promedio: {df_eps_hc['elapsed_time'].mean():.2f} segundos")

### Visualización de Resultados Hard-Core

In [ ]:
# Crear visualizaciones para Hard-Core
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Estimaciones vs tamaño del lattice
ax1 = axes[0, 0]
for eps in epsilon_values:
    df_eps_hc = df_hc[df_hc['epsilon'] == eps]
    ax1.semilogy(df_eps_hc['K'], df_eps_hc['estimate'], 
                'o-', label=f'ε={eps}', markersize=8)
ax1.set_xlabel('Tamaño del Lattice (K)')
ax1.set_ylabel('log(Número de Configuraciones)')
ax1.set_title('Crecimiento del Número de Configuraciones Hard-Core')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Número promedio de partículas
ax2 = axes[0, 1]
df_01_hc = df_hc[df_hc['epsilon'] == 0.1]
ax2.plot(df_01_hc['K'], df_01_hc['avg_particles'], 
        'go-', linewidth=2, markersize=8)
ax2.fill_between(df_01_hc['K'], 
                 df_01_hc['avg_particles'] - np.sqrt(df_01_hc['var_particles']),
                 df_01_hc['avg_particles'] + np.sqrt(df_01_hc['var_particles']),
                 alpha=0.3, color='green')
ax2.set_xlabel('Tamaño del Lattice (K)')
ax2.set_ylabel('Número Promedio de Partículas')
ax2.set_title('Partículas Promedio en Configuraciones Hard-Core (ε=0.1)')
ax2.grid(True, alpha=0.3)

# 3. Tiempo de ejecución
ax3 = axes[1, 0]
for eps in epsilon_values:
    df_eps_hc = df_hc[df_hc['epsilon'] == eps]
    ax3.plot(df_eps_hc['K'], df_eps_hc['elapsed_time'], 
            'o-', label=f'ε={eps}', markersize=8)
ax3.set_xlabel('Tamaño del Lattice (K)')
ax3.set_ylabel('Tiempo de Ejecución (segundos)')
ax3.set_title('Tiempo de Cómputo para Hard-Core')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Densidad de partículas
ax4 = axes[1, 1]
df_01_hc['density'] = df_01_hc['avg_particles'] / df_01_hc['k_vertices']
ax4.bar(df_01_hc['K'].astype(str), df_01_hc['density'], color='coral')
ax4.set_xlabel('Tamaño del Lattice (K)')
ax4.set_ylabel('Densidad (partículas/vértices)')
ax4.set_title('Densidad de Partículas en Hard-Core (ε=0.1)')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../resultados/hardcore_analisis.png', dpi=150, bbox_inches='tight')
plt.show()

print("Gráficas guardadas en resultados/hardcore_analisis.png")

### Comparación con Valores Exactos para Hard-Core

In [ ]:
print("=== Comparación Hard-Core con Conteo Exacto ===")
print("\nComparando aproximaciones con valores exactos para lattices pequeños:")
print("-" * 60)
print(f"{'K':^5} {'Exacto':^15} {'Aproximado':^15} {'Error (%)':^15}")
print("-" * 60)

hc_comparison_results = []

for K in [2, 3, 4, 5]:
    try:
        exact_hc = exact_hardcore_count(K)
        
        lattice = LatticeGraph(K)
        approx_hc = HardCoreApproximation(lattice, epsilon=0.05)
        estimate_hc, _ = approx_hc.approximate_count(verbose=False)
        
        error_hc = abs(estimate_hc - exact_hc) / exact_hc * 100 if exact_hc > 0 else 0
        
        print(f"{K:^5} {exact_hc:^15,} {estimate_hc:^15.0f} {error_hc:^15.2f}")
        
        hc_comparison_results.append({
            'K': K,
            'exact': exact_hc,
            'approximate': estimate_hc,
            'error_percent': error_hc
        })
        
    except Exception as e:
        print(f"{K:^5} {'N/A':^15} {'N/A':^15} {'N/A':^15}")

print("-" * 60)

df_hc_comparison = pd.DataFrame(hc_comparison_results)
if not df_hc_comparison.empty:
    print(f"\nError promedio: {df_hc_comparison['error_percent'].mean():.2f}%")
    print(f"Error máximo: {df_hc_comparison['error_percent'].max():.2f}%")
    print(f"Error mínimo: {df_hc_comparison['error_percent'].min():.2f}%")

## Guardar Resultados

In [ ]:
# Guardar todos los resultados en CSV
import os
os.makedirs('../resultados', exist_ok=True)

df_q.to_csv('../resultados/q_coloraciones_resultados.csv', index=False)
df_hc.to_csv('../resultados/hardcore_resultados.csv', index=False)

if not df_comparison.empty:
    df_comparison.to_csv('../resultados/q_coloraciones_comparacion.csv', index=False)

if not df_hc_comparison.empty:
    df_hc_comparison.to_csv('../resultados/hardcore_comparacion.csv', index=False)

print("Resultados guardados en la carpeta 'resultados/'")
print("\nArchivos generados:")
print("  • q_coloraciones_resultados.csv")
print("  • hardcore_resultados.csv")
print("  • q_coloraciones_comparacion.csv")
print("  • hardcore_comparacion.csv")
print("  • q_coloraciones_analisis.png")
print("  • hardcore_analisis.png")

## Resumen y Conclusiones

### Resultados Principales

1. **q-Coloraciones:**
   - Implementación exitosa del algoritmo basado en el Teorema 9.1
   - La condición $q > 2d^*$ es crítica para la convergencia
   - El tiempo de mezcla crece con $O(k \log k)$ como predice la teoría

2. **Modelo Hard-Core:**
   - El número de configuraciones crece exponencialmente con el tamaño del lattice
   - La densidad de partículas se estabiliza alrededor de 0.25-0.30 para lattices grandes
   - El algoritmo es eficiente incluso para lattices de tamaño moderado

3. **Precisión del Algoritmo:**
   - Con $\varepsilon = 0.1$, logramos errores típicamente menores al 20%
   - Mayor precisión requiere significativamente más tiempo de cómputo
   - Trade-off claro entre precisión y eficiencia computacional

### Observaciones Técnicas

- El número de simulaciones escala como $O(k^3/\varepsilon^2)$
- El tiempo de mezcla depende logarítmicamente de $q/(q-1)$
- Para lattices grandes (K > 10), el tiempo de cómputo se vuelve significativo

### Trabajo Futuro

- Implementar técnicas de reducción de varianza
- Explorar paralelización de las simulaciones
- Extender a otros tipos de grafos más allá de lattices